## Log of changes
- changes sent tokenizers with pysbd
    - think about change as change will affect processing in other languages

In [1]:
# import wikipediaapi

import os
import openai
import json
import numpy as np
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')
openai.api_key = json.load(open("/Users/shahules/openai-key.json"))['ikka']

/opt/anaconda3/envs/ragas/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def llm(prompt, **kwargs):
    response = openai.ChatCompletion.create(
        model=kwargs.get("model", "gpt-3.5-turbo"),
        messages=[{"role":"user","content":prompt}],
        temperature=kwargs.get("temperature", 0),
        top_p=kwargs.get("top_p", 1),
        frequency_penalty=kwargs.get("frequency_penalty", 0.0),
        presence_penalty=kwargs.get("presence_penalty", 0.0),
        max_tokens=kwargs.get("max_tokens", 500),
        n=kwargs.get("n", 1),
    )
    return response


In [3]:
message = [{"role": "system", "content": "You're a bot that answers any given question. If you dont know the exact answer make up one."},
{"role":"user", "content":"What were the temperatures and snowfall amounts during the cold snap in Afghanistan in January 2023, and how many people and livestock were affected?"}]

In [4]:
# llm(message)

In [5]:
wiki_wiki = wikipediaapi.Wikipedia(
    language="en", extract_format=wikipediaapi.ExtractFormat.WIKI
)

p_wiki = wiki_wiki.page("Black hole")


def get_page_section(page,chars=8000):
    all_text = ""
    p_wiki = wiki_wiki.page(page)    
    return p_wiki.text[:chars]


def get_cosine(page, backlinks):
    backlinks_vec = model.encode(backlinks)
    page_vec = model.encode([page]).reshape(1,-1)
    norm = np.linalg.norm(backlinks_vec,axis=1)*np.linalg.norm(page_vec,axis=1)
    cosine_sim = np.dot(backlinks_vec,page_vec.T).reshape(-1,)/norm
    return cosine_sim

def get_backlink_titles(page):
    p_wiki = wiki_wiki.page(page) 
    backlinks =  [i.title() for i in p_wiki.backlinks if ":" not in i.title()][:100]
    if len(backlinks)>1:
        c = get_cosine(page, backlinks)
        top_indices = c.argsort()[::-1][:10]
        return [backlinks[i] for i in top_indices if backlinks[i]!=page]
    return []
    
    

In [6]:
Question_gen = """
Given a wikipedia text generate a question that can be fully answered from the text of medium difficulty with long form answer seperated by \n\n from the given text.

Wikipedia Text:
The Eiffel Tower is a wrought-iron lattice tower located on the Champ de Mars in Paris, France. It was named after the engineer Gustave Eiffel, whose company designed and built the structure. Erected in 1889 as the entrance arch to the 1889 World's Fair, it has become a global cultural icon of France and one of the most recognizable structures in the world. The Eiffel Tower is the tallest structure in Paris and the most-visited paid monument in the world; millions of people ascend it every year. The tower stands at a height of 324 meters (1,063 feet) and was the tallest man-made structure in the world until the completion of the Chrysler Building in New York City in 1930.
The tower has three levels for visitors, with restaurants on the first and second levels. The third level observatory's upper platform is at 276 meters (906 feet) above the ground, the highest accessible to the public in the European Union. The tower has been featured in numerous films and TV shows, and its lighting is often modified to mark special events or holidays. Despite initial criticism from some of France's leading artists and intellectuals, the Eiffel Tower has become a global cultural icon of France and one of the most recognizable structures in the world.
Question: What is the height of the Eiffel Tower, and how does it compare to the height of the Chrysler Building in New York City?
\n\nAnswer:The height of the Eiffel Tower is 324 meters (1,063 feet). It was the tallest man-made structure in the world until the completion of the Chrysler Building in New York City in 1930. The Chrysler Building surpassed the Eiffel Tower's height and became the new tallest structure, reaching a height of 319 meters (1,046 feet) including its spire. However, it's important to note that the Eiffel Tower remains the tallest structure in Paris and still holds its title as the most-visited paid monument in the world. Despite losing its status as the tallest man-made structure globally, the Eiffel Tower's cultural significance and iconic status endure, attracting millions of visitors each year and leaving an indelible mark on the city of Paris and the world at large.
Wikipedia Text:\n{}
"""

In [7]:
from tqdm import tqdm
def generate_dataset(pages):
    data = []
    for page in tqdm(pages):
        try:
            text = get_page_section(page)
            message = [{"role": "user", "content":Question_gen.format(text)}]
            output = llm(message)
            question, answer_grounded = output['choices'][0]['message']['content'].split("\n\n")
            message = [{"role": "user", "content":question}]
            answer = llm(message)['choices'][0]['message']['content']
            data.append({
                "question":question,
                "grounded_answer":answer_grounded,
                "answer":answer,
                "context":text,
                "source":page,
            })
        except Exception as e:
            print(e)
    return data
        
        

https://en.wikipedia.org/wiki/2023#January

In [27]:
with open("/Users/shahules/belar/experimental/pages.txt", "r") as file:
    pages = file.read()


In [28]:
# output = llm(Question_gen.format(c.text))

In [29]:
# question, answer = output['choices'][0]['message']['content'].split("\n\n")

In [30]:
pages = pages.split('\n')[:-1]

In [31]:
pages = ["Volcanism on Venus","Pandemic prevention","Jupiter Icy Moons Explorer"]

In [32]:
data_sample = generate_dataset(pages)

100%|█████████████████████████████████████████████████████| 3/3 [00:54<00:00, 18.14s/it]


In [34]:
data = json.load(open("/Users/shahules/belar/experimental/ragas_wiki_eval.json"))
data.extend(data_sample)

In [35]:
len(data)

50

In [36]:
with open("ragas-eval-data.json",'w') as file:
    json.dump(data, file, indent=4)

In [50]:
"""
    get_recent_changes.py

    MediaWiki API Demos
    Demo of `RecentChanges` module: Get the three most recent changes with
    sizes and flags

    MIT License
"""

import requests

S = requests.Session()

URL = "https://en.wikipedia.org/w/api.php"

PARAMS = {
    "format": "json",
    "rcprop": "title|timestamp|tags",
    "list": "recentchanges",
    "action": "query",
    "rclimit": "1000",
    "rctype":"new"
}

R = S.get(url=URL, params=PARAMS)
DATA = R.json()

RECENTCHANGES = DATA['query']['recentchanges']
remove_tags = ["User:","Category:","Draft","User talk:","Talk:","Wikipedia:","Template talk:"]

for rc in RECENTCHANGES:
    title = str(rc['title'])
    timestamp = str(rc['timestamp'])
    if not any([x in title for x in remove_tags]) and "2021" not in timestamp:
        
        print(title)


Sorn-khiri Sriprachuap
Methane synthesis
1993–94 BCFC season
Hagawi
Populus mexicana
Gracie Lawrence
Gracie Bea Lawrence
Laura Cornelius
A. D. Macklin
Polyosma hirsuta
Maltzanella
Molina Seca
The Life of the Party (play)
Ramnagar Assembly constituency (disambiguation)
Blue Angel (Park Ji-yoon album
A.D. Macklin
Blanca Wiethüchter López
Frances Hughes
Gubad Ibadoghlu
2023 BWF World Championships – Men's doubles
Jesús Urzagasti Aguilera
1925 Reform Party (New Zealand) leadership election
Ana Pinho Rodrigues
Olympian 6
80th Venice Film Festival
2023 Venice Film Festival
2023 Venice International Film Festival
We Find the Bunyip
C-ya-laterrrr
Evan asano
13th Politburo Standing Committee of the Chinese Communist Party
Chase Blasi
Hateful speech
Template:WIR-280
Archaeology in Africa
Home (Secret Invasion)
Anil Thadani
Love Me a Little
Wayne Stevens (disambiguation)
Wayne Osborne (disambiguation)
Wayne Hughes (disambiguation)
Wayne Hammond (disambiguation)
Thai Empire (disambiguation)
Oxygen

In [286]:
data = json.load(open("ragas-data.json"))

In [94]:
evading = ["I'm sorry", "as an AI language model"]

In [99]:
topics_regen = [item for item in data if any(x.lower() in item['answer'].lower() for x in evading)]

In [101]:
message = [{"role": "system", "content": "You're a bot that answers any given question. If you dont know the exact answer make up one."},]

In [105]:
for item in topics_regen:
    message = [{"role": "system", "content": "You're a bot that answers any given question. If you dont know the exact answer make up one."},]
    message.append({"role":"user","content":item['question']})
    output = llm(message)
    item['answer'] = output['choices'][0]['message']['content']

In [42]:
for item in tqdm(data):
    titles = get_backlink_titles(item["source"])
    context = [get_page_section(item,chars=1000) for item in titles[:2]]
    context.insert(0,item['context'])
    item["context_retrieved"] = context

100%|███████████████████████████████████████████████████| 50/50 [01:23<00:00,  1.67s/it]


In [119]:
with open("ragas_wiki_evalv1.json",'w') as file:
    json.dump(data, file, indent=4)

In [56]:
Dataset.from_json("ragas-eval-data.json").push_to_hub("explodinggradients/wiki-eval")

Found cached dataset json (/Users/shahules/.cache/huggingface/datasets/json/default-d425d908fb97025a/0.0.0)
Creating parquet from Arrow format: 100%|█████████████████| 1/1 [00:00<00:00, 50.02ba/s]

Pushing dataset shards to the dataset hub: 100%|██████████| 1/1 [00:06<00:00,  6.27s/it]


## Generate low relevancy answers

In [3]:
from datasets import load_dataset

In [4]:
irr_answer = """
Answer the given question partially.
Question:{}
"""

In [12]:
dataset = load_dataset("explodinggradients/wiki-eval")

Extracting data files: 100%|████████| 1/1 [00:00<00:00, 389.52it/s]


Dataset parquet downloaded and prepared to /Users/shahules/.cache/huggingface/datasets/explodinggradients___parquet/explodinggradients--wiki-eval-f0df84e235efd078/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


100%|███████████████████████████████| 1/1 [00:00<00:00, 224.16it/s]


In [29]:
answers = []
for item in dataset['train']:
    question = item['question']
    message = []
    message.append({"role":"user","content":irr_answer.format(question)})
    while True:
        try:
            answer = llm(message)['choices'][0]['message']['content']
        except Exception as e:
            continue
        break
    answers.append(answer)


In [31]:
dataset = dataset["train"].add_column("partial_answer",answers)

In [32]:
dataset.push_to_hub("explodinggradients/wiki-eval")

Pushing dataset shards to the dataset hub:   0%| | 0/1 [00:00<?, ?i
Creating parquet from Arrow format: 100%|█| 1/1 [00:00<00:00, 79.01

Upload 1 LFS files: 100%|████████████| 1/1 [00:06<00:00,  6.60s/it]
Pushing dataset shards to the dataset hub: 100%|█| 1/1 [00:07<00:00
Deleting unused files from dataset repository: 100%|█| 1/1 [00:00<0
Updating downloaded metadata with the new split.


In [231]:
with open("ragas_wiki_evalv1.json",'w') as file:
    json.dump(data, file, indent=4)

In [17]:
dataset = load_dataset("explodinggradients/wiki-eval")

Found cached dataset parquet (/Users/shahules/.cache/huggingface/datasets/explodinggradients___parquet/explodinggradients--wiki-eval-80d4ef132547f2df/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100%|████████████████████████████████████████████████████| 1/1 [00:00<00:00, 241.32it/s]


In [18]:
dataset = dataset.select_columns(["question","source","answer","grounded_answer","answer_relevancy_v1","context_v1","context_v2"])

In [19]:
dataset = dataset.rename_columns({"answer":"ungrounded_answer","answer_relevancy_v1":"poor_answer"})

In [20]:
dataset.push_to_hub("explodinggradients/WikiEval")

Pushing split train to the Hub.
Creating parquet from Arrow format: 100%|█████████████████| 1/1 [00:00<00:00, 97.16ba/s]

Deleting unused files from dataset repository: 100%|██████| 1/1 [00:00<00:00,  2.49it/s]


In [80]:
import json
def record_score(scores,name,filename='ragas_promptscoring'):
    data = json.load(open(f"{filename}.json"))
    data[name] = scores
    with open(f"{filename}.json",'w') as file:
        json.dump(data,file,indent=4)
        
def read_score(score_name):
    data = json.load(open("ragas_abblation.json"))
    return data[score_name]

## Ablation studies

In [81]:
from datasets import load_dataset,Dataset
import numpy as np

In [112]:
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
)
from ragas import evaluate
from ragas.metrics.context_relevance import ContextRelevancy

In [13]:
context_relevancy = ContextRelevancy(strictness=1)

In [8]:
wikieval = load_dataset("explodinggradients/WikiEval")

Found cached dataset parquet (/Users/shahules/.cache/huggingface/datasets/explodinggradients___parquet/explodinggradients--WikiEval-3b60abf6f625ac40/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)
100%|█████████████████████████████████████████████████████| 1/1 [00:00<00:00, 35.36it/s]


In [7]:
wikieval = wikieval['train'].rename_columns({"grounded_answer":"answer","context_v1":"contexts"})

In [19]:
wikieval1 = wikieval.select_columns(['answer', 'question','contexts'])

In [20]:
results = evaluate(dataset=wikieval1,metrics=[context_relevancy,faithfulness,answer_relevancy])

evaluating with [context_relevancy]


100%|█████████████████████████████████████████████████████| 4/4 [04:54<00:00, 73.72s/it]


evaluating with [faithfulness]


100%|████████████████████████████████████████████████████| 4/4 [08:41<00:00, 130.45s/it]


evaluating with [answer_relevancy]


100%|█████████████████████████████████████████████████████| 4/4 [02:01<00:00, 30.46s/it]


In [22]:
wikieval1.select(range(0,20)).to_csv("/Users/shahules/Downloads/wikieval_sample20.csv")

Creating CSV from Arrow format: 100%|█████████████████████| 1/1 [00:00<00:00, 10.41ba/s]


87063

In [23]:
results

{'ragas_score': 0.6130, 'context_relevancy': 0.3521, 'faithfulness': 0.9865, 'answer_relevancy': 0.9613}

In [24]:
results.to_pandas()

,answer,question,contexts,context_relevancy,faithfulness,answer_relevancy
0,Answer: The PSLV-C56 mission is scheduled to b...,Question: When is the scheduled launch date an...,[The PSLV-C56 is the 58th mission of Indian Sp...,0.400000,1.000000,0.942336
1,Answer: The objective of the Uzbekistan-Afghan...,Question: What is the objective of the Uzbekis...,[The Uzbekistan–Afghanistan–Pakistan Railway P...,1.000000,1.000000,0.949762
2,Answer: PharmaCann was founded in 2014 by Theo...,Question: When was PharmaCann founded and what...,"[Found in 2014 by Theodore Scott, PharmaCann i...",0.333333,1.000000,0.963253
3,Answer: Christopher Nolan directed the film Op...,Question: Who directed the film Oppenheimer an...,[Oppenheimer is a 2023 biographical thriller f...,0.038462,1.000000,0.984808
4,"Answer: Theranostics, also known as theragnost...",Question: What is theranostics and how does it...,"[Theranostics, also known as theragnostics, is...",0.457143,1.000000,0.981180
5,Answer: The human climate niche refers to the ...,Question: What is the human climate niche and ...,[The human climate niche is the ensemble of cl...,0.666667,1.000000,0.981146
6,Answer: Dasypoda radchenkoi belongs to the gen...,Question: What is the taxonomy of Dasypoda rad...,"[Dasypoda radchenkoi, also known as Radchenko'...",0.200000,1.000000,0.969072
7,Answer: The main product of Fremantle Octopus ...,Question: What is the main product of Fremantl...,[Fremantle Octopus is an Australian octopus fi...,0.545455,1.000000,0.971285
8,Answer: The Managing Director of FoodFutureCo ...,Question: Who is the Managing Director of Food...,[FoodFutureCo is a scale-up accelerator for pu...,0.545455,1.000000,0.977580
9,Answer: The purpose of designing and building ...,Question: What was the purpose of designing an...,[The Fiat Ecobasic is a concept car designed b...,0.333333,1.000000,0.981272


### Concat reannoated samples with all samples

In [93]:
wikieval_og = load_dataset("explodinggradients/WikiEval")
wikieval_local = Dataset.from_csv("/Users/shahules/Downloads/wikieval_sample20 - wikieval_sample20.csv.csv")
wikieval_poor_answer = Dataset.from_csv("/Users/shahules/Downloads/wikieval_sample20 - poor_answer.csv")

Found cached dataset parquet (/Users/shahules/.cache/huggingface/datasets/explodinggradients___parquet/explodinggradients--WikiEval-3b60abf6f625ac40/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)
100%|████████████████████████████████████████████████████| 1/1 [00:00<00:00, 243.18it/s]
Found cached dataset csv (/Users/shahules/.cache/huggingface/datasets/csv/default-7607020e99f8dfe0/0.0.0)


Extracting data files: 100%|█████████████████████████████| 1/1 [00:00<00:00, 204.24it/s]
                                                        

Dataset csv downloaded and prepared to /Users/shahules/.cache/huggingface/datasets/csv/default-6323ab1c8fd55177/0.0.0. Subsequent calls will reuse this data.


In [94]:
wikieval_og = wikieval_og.rename_columns({"grounded_answer":"answer"})

In [95]:
wikieval = wikieval_og['train'].select(range(0,20))

In [96]:
wikieval_local = wikieval_local.add_column('poor_answer', wikieval['poor_answer'])

In [97]:
wikieval_local = wikieval_local.add_column('source', wikieval['source'])

In [98]:
wikieval_local = wikieval_local.rename_columns({'contexts':'context_v1'})
wikieval_local = wikieval_local.map(lambda x : {"context_v1":eval(x['context_v1'])})
wikieval_local = wikieval_local.map(lambda x : {"context_v2":eval(x['context_v2'])})

Loading cached processed dataset at /Users/shahules/.cache/huggingface/datasets/csv/default-7607020e99f8dfe0/0.0.0/cache-b666c3bbfd9a9a68.arrow
Loading cached processed dataset at /Users/shahules/.cache/huggingface/datasets/csv/default-7607020e99f8dfe0/0.0.0/cache-51de8ac189b2e2d9.arrow


In [99]:
from datasets import concatenate_datasets

In [100]:
wikieval_latest = concatenate_datasets([wikieval_local,wikieval_og['train'].select(range(20,50))])

- Update poor answer

In [109]:
wikieval_latest = wikieval_latest.remove_columns(['poor_answer'])

In [111]:
wikieval_latest = wikieval_latest.add_column('poor_answer',wikieval_poor_answer['poor_answer'])

In [131]:
wikieval_latest.push_to_hub("explodinggradients/WikiEval")

Deleting unused files from dataset repository: 100%|██████| 1/1 [00:00<00:00,  3.38it/s]
Updating downloaded metadata with the new split.


## reannotates sample
Test assumptions
1. Ragas scores ranked correctly
2. GPT3.5 scores 
3. GPT3.5 rankings

In [113]:
from datasets import Dataset
# from ragas.metrics.context_relevance import sent_tokenize as ragas_tokenizer

In [116]:
column_map = {
    "question":"question",
#     "contexts":"context_v2",
    "answer":"poor_answer",
}

In [117]:
results = evaluate(dataset=wikieval_latest,
                   metrics=[answer_relevancy],column_map=column_map)

evaluating with [answer_relevancy]


100%|█████████████████████████████████████████████████████| 4/4 [02:17<00:00, 34.42s/it]


In [257]:
results

{'ragas_score': 0.3263, 'context_relevancy': 0.1971, 'answer_relevancy': 0.9478}

In [118]:
results.to_pandas()['answer_relevancy'].values.tolist()

[0.9467473478661362,
 0.9637259131522917,
 0.9443463300116206,
 0.9402534786633633,
 0.9218596588819808,
 0.9285405166500519,
 0.9487620446850666,
 0.9506784002084028,
 0.941689354840754,
 0.9588847769982651,
 0.9318374062221277,
 0.9516390195713166,
 0.9483163635481464,
 0.9603426519082335,
 0.9266954346498673,
 0.9575183534547804,
 0.9801361309620589,
 0.8827392620210031,
 0.9381805505078088,
 0.9464205724284924,
 0.938312268188313,
 0.9341558598962424,
 0.9553525482187063,
 0.9457937101525687,
 0.9635201054966758,
 0.986305022132583,
 0.9472364344213897,
 0.8825884294134566,
 0.9796230393529424,
 0.9265061660514919,
 0.9693077277693484,
 0.9546205206771519,
 0.9561355788062716,
 0.9668649935237948,
 0.8428095549101032,
 0.9439265136674045,
 0.9473495611768238,
 0.9525520918708642,
 0.9578299924585187,
 0.897650469966222,
 0.9478177575220471,
 0.9522007994869987,
 0.9343958852735169,
 0.9155273173058056,
 0.9565743557995666,
 0.966526310557822,
 0.9555808438909955,
 0.958087176669835

In [21]:
record_score(results.to_pandas()['faithfulness'].values.tolist(),"ungrounded_faithfulness_v1")

In [22]:
results.to_pandas()['faithfulness'].values.tolist()

[0.5,
 0.5,
 0.5,
 0.5,
 0.8,
 0.75,
 0.6,
 0.0,
 0.33333333333333337,
 0.0,
 0.5,
 0.0,
 0.5,
 0.6666666666666667,
 0.33333333333333337,
 0.5,
 0.33333333333333337,
 0.6,
 0.6666666666666667,
 0.0]

In [16]:
results.to_pandas()['faithfulness'].values.tolist()

[1.0,
 0.6666666666666667,
 1.0,
 0.5,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.875,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0]

In [21]:
comp_matrix = np.vstack([read_score("gt_context_v1"),read_score("context_v2_v1")])

In [26]:
comp_matrix[0,:] >= comp_matrix[1,:]

array([ True,  True,  True, False,  True,  True,  True,  True,  True,
        True,  True, False,  True,  True,  True,  True,  True,  True,
        True,  True])

In [28]:
np.where(comp_matrix[0,:] < comp_matrix[1,:])

(array([ 3, 11]),)

In [23]:
len(ragas_tokenizer(wikieval1[1]['contexts'][0]))

10

In [83]:
from nltk.tokenize import sent_tokenize

In [88]:
len(sent_tokenize(wikieval1[1]['contexts'][0]))

9

In [91]:
import pysbd
seg = pysbd.Segmenter(language="en", clean=False)
len(seg.segment(wikieval1[1]['contexts'][0]))

10

#### Annotate samples using gpt direct scoring

In [68]:
metric_name = "answer_relevancy"
definition = """answer relevancy: refers to the degree to which a response directly addresses and is appropriate for a given question or context. Penalize incomplete or reduntant information in the answer"""

In [72]:
prompt = """
{definition}
Given question and answer, assign a score for {metric_name} in the range 0-10.
question:\n{question}
answer:\n{answer}
{metric_name} score:
"""

In [80]:
item = wikieval[2]

In [83]:
prompt_input = prompt.format(definition=definition,metric_name=metric_name,
                             question=item['question'],
                             answer=item['grounded_answer'])

In [84]:
llm(prompt_input)

<OpenAIObject chat.completion id=chatcmpl-7yHqQgOC2q8W4x876LorxX9Im4bP3 at 0x7f78d1192570> JSON: {
  "id": "chatcmpl-7yHqQgOC2q8W4x876LorxX9Im4bP3",
  "object": "chat.completion",
  "created": 1694602630,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "8"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 115,
    "completion_tokens": 1,
    "total_tokens": 116
  }
}

### Annotate using gpt ranking

In [1]:
prompt = """
{definition}
Given question and two contexts , output rank of each context based on {metric_name}
question:\n{question}
context1:\n{context1}
context2:\n{context2}
output ranks:
"""

## Run in batches

In [51]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.chat_models import ChatOpenAI
from ragas.metrics.llms import generate

In [52]:
llm =  ChatOpenAI(model_name="gpt-3.5-turbo-16k")

In [218]:
def run_batch(dataset,prompt,llm,batch_size=15):
    
    results = []
    for i in range(0,len(dataset),batch_size):
        prompts = []
        ds = dataset.select(range(i,min(len(dataset),i+batch_size)))
        answer1, answer2, question = ds["answer"], ds["poor_answer"], ds["question"]
        for q, a1, a2 in zip(question, answer1, answer2):
#                 c_str: str = "\n".join(c)
                human_prompt = prompt.format(answer1=a2, answer2=a1, question=q)
                prompts.append(ChatPromptTemplate.from_messages([human_prompt]))  
        result = generate(prompts, llm)
        result = [output[0].text for output in result.generations]
        results.extend(result)
        
    return results

    

## prompt scoring

`<column_name>_<metric_name>`

In [219]:
faithfulness_prompt = """
Faithfulness measures the information consistency of the answer against the given context. Any claims that are made in the answer that cannot be deduced from context should be penalized
Given an answer and context, assign a score for faithfulness in the range 0-10.

answer:\n{answer}
context:\n{context}
faithfulness score:
"""

context_relevancy_prompt = """
Context Relevancy measures how relevant retrieved contexts are to the question. Ideally, the context should only contain information necessary to answer the question. The presence of redundant information in the context is penalized.
Given an question and context, assign a score for context relevancy in the range 0-10.

question:\n{question}
context:\n{context}
context relevancy score:
"""

answer_relevancy_prompt = """
Answer Relevanc measures the degree to which a response directly addresses and is appropriate for a given question. It penalizes the present of redundant information or incomplete answers given a question.
Given an question and answer, assign a score for answer relevancy in the range 0-10.

question:\n{question}
answer:\n{answer}
answer relevancy score:
"""

In [222]:
# results = run_batch(wikieval_latest.select(range(0,50)), prompt=faithfulness_prompt, llm=llm)

In [223]:
# results = [float(i) for i in results]

## Ranking

`We always provide the poor answer first and best answer last `

In [225]:
faithfulness_prompt = """
Faithfulness measures the information consistency of the answer against the given context. Any claims that are made in the answer that cannot be deduced from context should be penalized
Given context and two different answers , rank each answer based on faithfulness.

context:Albert Einstein was a German-born theoretical physicist, widely held to be one of the greatest and most influential scientists of all time.
answer1:Albert Einstein was born in Germany.
answer2:Albert Einstein was born in Spain.
output:answer1\n\nanswer2

context:\n{context}
answer1:\n{answer1}
answer2:\n{answer2}
output:"""


context_relevancy_prompt = """
Context Relevancy measures how relevant retrieved contexts are to the question. Ideally, the context should only contain information necessary to answer the question. The presence of redundant information in the context is penalized.
Given an question and context, rank each context based on context relevancy.

question:When was Einstein born?
context1: Einstein was a German-born theoretical physicist. He was born in 14 March 1879.
context2: Albert Einstein was a German-born theoretical physicist. Widely held to be one of the greatest and most influential scientists of all time. He was born in 14 March 1879.
output:context1\n\ncontext2

question:\n{question}
context1:\n{context1}
context2:\n{context2}
output:"""



answer_relevancy_prompt = """
Answer Relevancy measures the degree to which a response directly addresses and is appropriate for a given question. It penalizes the present of redundant information or incomplete answers given a question.
Given an question and answer,  rank each answer based on Answer Relevancy.

question: Where was Einstein born?
answer1:Albert Einstein was born in Germany.
answer2:Albert Einstein was born in Spain and lived in America.
output:answer1\n\nanswer2

question:\n{question}
answer1:\n{answer1}
answer2:\n{answer2}
output:
"""

In [226]:
results = run_batch(wikieval_latest.select(range(0,50)), prompt=answer_relevancy_prompt, llm=llm)

In [233]:
# results = [i.split('\n\n') for i in results]
answer1_ranks = [i.index('answer1') if 'answer1' in i else 0  for i in results]
answer2_ranks = [i.index('answer2') if 'answer1' in i else 0  for i in results]

## Kendall corr

In [8]:
from scipy.stats import kendalltau,spearmanr
import pandas as pd
import numpy as np

In [265]:
def get_tau(x,y):
    return [kendalltau(x1, y1).statistic for x1,y1 in zip(x,y)]


In [293]:
kendalltau([[1.0,0.5],[0.8,0.3]],[[1,0],[1,0]])

SignificanceResult(statistic=0.8164965809277261, pvalue=0.12133525035848211)

In [296]:
np.array([[1.0,0.5],[0.8,0.3]]).flatten()

array([1. , 0.5, 0.8, 0.3])

In [121]:
df = pd.read_csv("/Users/shahules/Downloads/RAGAS-paper - ragas.csv")

In [127]:
cols = [('answer_faithfulness', 'ungrounded_answer_faithfulness'),
        ('context_v1_context_relevancy', 'context_v2_context_relevancy'),
       ('answer_answer_relevancy', 'poor_answer_answer_relevancy'),
       ]

In [128]:
def correlation_report(df):
    for col in cols:
        x = df[col[0]].values.tolist()
        y = df[col[1]].values.tolist()
        scores = list(zip(x,y))
        target = [[1,0]] * 50
        print((get_tau(scores,target)))
        
def accuracy_report(df):
    for col in cols:
        x = df[col[0]].values.tolist()
        y = df[col[1]].values.tolist()
        scores = list(zip(x,y))
#         scores = [list(x) for x in scores]
        scores = [list(np.argsort(x)) if x[0]!=x[1] else [0,0] for x in scores]
        target = [[1,0]] * 50
        print(sum([s==t for s,t in zip(scores,target)])/len(target))

In [129]:
accuracy_report(df)

0.92
0.76
0.7


In [ ]:
zip([1,2,3],[])

In [267]:
get_tau([[1,2],[1,3]],[[2,3],[1,3]])

[1.0, 1.0]

In [292]:
spearmanr([[1,2],[1,3]],[[1,2],[1,3]])

SignificanceResult(statistic=nan, pvalue=nan)